In [ ]:
!pip install langchain
!pip install openai
!pip install langchain-community langchain
!pip install langgraph
!pip install -U langchain langchain-openai

In [2]:
import os
os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 'LANGCHAIN_API_KEY'
os.environ["LANGCHAIN_PROJECT"] = "Langgraph_1"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [29]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver

In [30]:
class HumanInLoop(TypedDict):
  step1: str
  step2: str
  input: str

In [31]:
import time

def step1_execution(state: HumanInLoop) -> HumanInLoop:
  print("Step 1 executed")
  return {"step1": "executed", 'input':state["input"]}

def step2_execution(state: HumanInLoop) -> HumanInLoop:
  print("Step 2 halted #######################")
  time.sleep(30)
  return {"step1": "executed", 'input':state["input"]}

def step3_execution(state: HumanInLoop) -> HumanInLoop:
    print("✅ Step 3 executed")
    return {"done": True}

In [32]:
# 3. Build the graph
builder = StateGraph(HumanInLoop)
builder.add_node("step_1", step1_execution)
builder.add_node("step_2", step2_execution)
builder.add_node("step_3", step3_execution)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [33]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
Step 1 executed
Step 2 halted #######################
❌ Kernel manually interrupted (crash simulated).


In [34]:
graph.get_state({
    "configurable": {"thread_id": 'thread-1'}
    })

StateSnapshot(values={'step1': 'executed', 'input': 'start'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d49d-6a70-8001-51fac2dfbd30'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-17T03:59:19.814805+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d494-6c08-8000-b34165467937'}}, tasks=(PregelTask(id='6da76555-ad6f-9917-d0f8-226c12f2fc7b', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [35]:
list(graph.get_state_history({
    "configurable": {"thread_id": 'thread-1'}
    }))

[StateSnapshot(values={'step1': 'executed', 'input': 'start'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d49d-6a70-8001-51fac2dfbd30'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-17T03:59:19.814805+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d494-6c08-8000-b34165467937'}}, tasks=(PregelTask(id='6da76555-ad6f-9917-d0f8-226c12f2fc7b', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'input': 'start'}, next=('step_1',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d494-6c08-8000-b34165467937'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-17T03:59:19.811158+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '',

In [36]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
Step 2 halted #######################
✅ Step 3 executed

✅ Final State: {'step1': 'executed', 'input': 'start'}


In [37]:
graph.get_state({
    "configurable": {"thread_id": 'thread-1'}
    })

StateSnapshot(values={'step1': 'executed', 'input': 'start'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1ec-ec1c-63d6-8003-62ef20afed56'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-17T04:01:09.652561+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1ec-ec16-62c8-8002-fd2d4fbef167'}}, tasks=(), interrupts=())

In [38]:
list(graph.get_state_history({
    "configurable": {"thread_id": 'thread-1'}
    }))

[StateSnapshot(values={'step1': 'executed', 'input': 'start'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1ec-ec1c-63d6-8003-62ef20afed56'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-17T04:01:09.652561+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1ec-ec16-62c8-8002-fd2d4fbef167'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'step1': 'executed', 'input': 'start'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1ec-ec16-62c8-8002-fd2d4fbef167'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-17T04:01:09.650052+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f07b1e8-d49d-6a70-8001-51fac2dfbd30'}}, tasks=(PregelTask(id='746d3303-bd00-9c8f-16fe-498c81dbccd2', name='step_3', path=('